In [19]:
import cv2
import time
import math
import mediapipe as mp
import numpy as np
import hand_track as ht
import autopy

In [20]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [21]:
timeP = 0
vol_bar=0
vol_value=0

In [22]:
cap = cv2.VideoCapture(0)
tracker = ht.hand_tracker(max_num_hands=1, detection_conf=0.6, tracking_conf=0.7)

In [23]:
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
vol_range = volume.GetVolumeRange()
min_vol = vol_range[0]
max_vol = vol_range[1]
#volume.SetMasterVolumeLevel(-20.0, None)

volume range is from -65 to 0
finger range is from 15 to 112

In [24]:
while True:
    ret, frame = cap.read()
    frame = tracker.detection(frame)
    lm_coord, bbox = tracker.position(frame)
    
    if len(lm_coord)!=0:
        idx_x, idx_y = lm_coord[8][1], lm_coord[8][2]
        mid_x, mid_y = lm_coord[12][1], lm_coord[12][2]
        
        cv2.circle(frame, (idx_x, idx_y), 10, (255,255,255), cv2.FILLED)
        cv2.circle(frame, (mid_x, mid_y), 10, (255,255,255), cv2.FILLED)
        cv2.line(frame, (idx_x, idx_y), (mid_x, mid_y), (255,0,255), 4)
    
        fingers = tracker.fingers_pos()
        
        if fingers[1]==1 and fingers[2]==1 :
            length, idx2mid_line = tracker.distance(8, 12)
            
            vol = np.interp(length, [20, 115], [min_vol, max_vol])
            vol_bar = np.interp(length, [20, 115], [60, 550])
            vol_value = np.interp(length, [20, 115], [0, 100])
            
            volume.SetMasterVolumeLevel(vol, None)
            #print(vol, length)

        cv2.rectangle(frame, (60, 400), (550, 415), (255,0,0), 3)
        cv2.rectangle(frame, (60, 400), (int(vol_bar), 415), (255,0,0), cv2.FILLED)
        cv2.putText(frame, str(int(vol_value)), (555,420), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,0), 3)
        
    timeC = time.time()
    fps = 1/(timeC-timeP)
    timeP = timeC
    
    cv2.putText(frame, str(int(fps)), (5,50), cv2.FONT_HERSHEY_DUPLEX, 3, (0,255,0), 3)
    cv2.imshow('frame', frame)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"):
        break
cv2.destroyAllWindows()